# phase II : outliers detection and removal

In [2]:
import pandas as pd
import numpy as np


In [24]:
data = pd.read_excel('data_phase1.xlsx')
data = data.drop(['Unnamed: 0'], axis=1)
data

,location,size,total_sqft,bath,price,BHK,price_per_sqrt
0,Electronic City Phase II,2 BHK,1056.0,2,39.07,2,3699.810606
1,Chikka Tirupathi,4 Bedroom,2600.0,5,120.00,4,4615.384615
2,Uttarahalli,3 BHK,1440.0,2,62.00,3,4305.555556
3,Lingadheeranahalli,3 BHK,1521.0,3,95.00,3,6245.890861
4,Kothanur,2 BHK,1200.0,2,51.00,2,4250.000000
...,...,...,...,...,...,...,...
13315,Whitefield,5 Bedroom,3453.0,4,231.00,5,6689.834926
13316,other,4 BHK,3600.0,5,400.00,4,11111.111111
13317,Raja Rajeshwari Nagar,2 BHK,1141.0,2,60.00,2,5258.545136
13318,Padmanabhanagar,4 BHK,4689.0,4,488.00,4,10407.336319


In [25]:
data.describe()


,total_sqft,bath,price,BHK,price_per_sqrt
count,13274.000000,13320.000000,13320.000000,13320.000000,1.327400e+04
mean,1559.626694,2.688814,112.565627,2.802778,7.907501e+03
std,1238.405258,1.338754,148.971674,1.294496,1.064296e+05
min,1.000000,1.000000,8.000000,1.000000,2.678298e+02
25%,1100.000000,2.000000,50.000000,2.000000,4.266865e+03
50%,1276.000000,2.000000,72.000000,3.000000,5.434306e+03
75%,1680.000000,3.000000,120.000000,3.000000,7.311746e+03
max,52272.000000,40.000000,3600.000000,43.000000,1.200000e+07


In [26]:
# we can clearly see the outliers here

(data['total_sqft']/data['BHK']).describe()

count    13274.000000
mean       575.074878
std        388.205175
min          0.250000
25%        473.333333
50%        552.500000
75%        625.000000
max      26136.000000
dtype: float64

In [27]:
# we will remove those data which has total sqft/BHk < 300

data = data[data['total_sqft']/data['BHK'] >= 300]

In [28]:
data.describe()

,total_sqft,bath,price,BHK,price_per_sqrt
count,12530.000000,12530.000000,12530.000000,12530.000000,12530.000000
mean,1594.564544,2.559537,111.382401,2.650838,6303.979357
std,1261.271296,1.077938,152.077329,0.976678,4162.237981
min,300.000000,1.000000,8.440000,1.000000,267.829813
25%,1116.000000,2.000000,49.000000,2.000000,4210.526316
50%,1300.000000,2.000000,70.000000,3.000000,5294.117647
75%,1700.000000,3.000000,115.000000,3.000000,6916.666667
max,52272.000000,16.000000,3600.000000,16.000000,176470.588235


In [29]:
data.shape

(12530, 7)

In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12530 entries, 0 to 13319
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   location        12530 non-null  object 
 1   size            12530 non-null  object 
 2   total_sqft      12530 non-null  float64
 3   bath            12530 non-null  int64  
 4   price           12530 non-null  float64
 5   BHK             12530 non-null  int64  
 6   price_per_sqrt  12530 non-null  float64
dtypes: float64(3), int64(2), object(2)
memory usage: 783.1+ KB


In [30]:
# we can see the obious outliers

data['price_per_sqrt'].describe()

count     12530.000000
mean       6303.979357
std        4162.237981
min         267.829813
25%        4210.526316
50%        5294.117647
75%        6916.666667
max      176470.588235
Name: price_per_sqrt, dtype: float64

In [32]:
# we can build a function to remove the outliers

def remove_sqft_outliers(df):
    df_output = pd.DataFrame()
    for key, subdf in df.groupby('location'):
        m = np.mean(subdf.price_per_sqrt)
        
        st = np.std(subdf.price_per_sqrt)
        
        df_gen = subdf[(subdf.price_per_sqrt > (m-st)) & (subdf.price_per_sqrt <= (m+st))]
        
        df_output = pd.concat([df_output, df_gen], ignore_index = True)
        
    return df_output
    
data = remove_sqft_outliers(data)
data.describe()

,total_sqft,bath,price,BHK,price_per_sqrt
count,10301.000000,10301.000000,10301.000000,10301.000000,10301.000000
mean,1508.440608,2.471702,91.286372,2.574896,5659.062876
std,880.694214,0.979449,86.342786,0.897649,2265.774749
min,300.000000,1.000000,10.000000,1.000000,1250.000000
25%,1110.000000,2.000000,49.000000,2.000000,4244.897959
50%,1286.000000,2.000000,67.000000,2.000000,5175.600739
75%,1650.000000,3.000000,100.000000,3.000000,6428.571429
max,30400.000000,16.000000,2200.000000,16.000000,24509.803922


In [33]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10301 entries, 0 to 10300
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   location        10301 non-null  object 
 1   size            10301 non-null  object 
 2   total_sqft      10301 non-null  float64
 3   bath            10301 non-null  int64  
 4   price           10301 non-null  float64
 5   BHK             10301 non-null  int64  
 6   price_per_sqrt  10301 non-null  float64
dtypes: float64(3), int64(2), object(2)
memory usage: 563.5+ KB


In [34]:
# bhk outliner removal 

def bhk_outliner_removal(df):
    exclude_indices = np.array([])
    for loc, loc_df in df.groupby('location'):
        bhk_stats = {}
        for bhk, bhk_df in loc_df.groupby('BHK'):
            bhk_stats[bhk] = {
                'mean' : np.mean(bhk_df.price_per_sqrt),
                'std' : np.std(bhk_df.price_per_sqrt),
                'count' : bhk_df.shape[0]
            }
            
        for bhk, bhk_df in loc_df.groupby('BHK'):
            stats = bhk_stats.get(bhk-1)
            if stats and stats['count']>5:
                exclude_indices = np.append(exclude_indices, bhk_df[bhk_df.price_per_sqrt<(stats['mean'])].index.values)
                
    return df.drop(exclude_indices, axis='index')

In [35]:
data = bhk_outliner_removal(data)
data.shape

(7361, 7)

In [37]:
# removing size and price per sqft( which we used to indentify outliers)

data = data.drop(['size', 'price_per_sqrt'], axis=1)
data

,location,total_sqft,bath,price,BHK
0,1st Block Jayanagar,2850.0,4,428.0,4
1,1st Block Jayanagar,1630.0,3,194.0,3
2,1st Block Jayanagar,1875.0,2,235.0,3
3,1st Block Jayanagar,1200.0,2,130.0,3
4,1st Block Jayanagar,1235.0,2,148.0,2
...,...,...,...,...,...
10292,other,1200.0,2,70.0,2
10293,other,1800.0,1,200.0,1
10296,other,1353.0,2,110.0,2
10297,other,812.0,1,26.0,1


In [38]:
data.describe()

,total_sqft,bath,price,BHK
count,7361.000000,7361.000000,7361.000000,7361.000000
mean,1496.942529,2.448173,99.093113,2.500611
std,865.781990,1.011515,93.336841,0.929312
min,300.000000,1.000000,10.000000,1.000000
25%,1096.000000,2.000000,50.000000,2.000000
50%,1260.000000,2.000000,73.200000,2.000000
75%,1680.000000,3.000000,113.000000,3.000000
max,30000.000000,16.000000,2200.000000,16.000000


In [39]:
data.to_excel('cleaned_data.xlsx')